In [ ]:
import pickle
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)
np.random.RandomState(42)

import time
import datetime as dt
#import statsmodels.api as sm
from scipy import stats
#from scipy.stats import norm, skew, kurtosis
import seaborn as sns

#from sklearn.linear_model import LinearRegression
#from sklearn.svm import SVR
#from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV,cross_val_score 
from sklearn import preprocessing
#from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, AdaBoostRegressor
#from xgboost.sklearn import XGBRegressor
#from sklearn.metrics import mean_absolute_error,r2_score, mean_squared_log_error,mean_squared_error, make_scorer

In [ ]:
data = pd.read_parquet(Path("data") / "train.parquet")

In [ ]:
data1 = pd.read_parquet(Path("data") / "test.parquet")

In [ ]:
df1 = pd.read_csv('2020.csv')

In [ ]:
df2 = pd.read_csv('2021.csv')

In [ ]:
public_holi20 = ["2020-01-01",'2020-04-10','2020-04-13','2020-05-01','2020-05-08','2020-05-21','2020-05-31','2020-06-01','2020-07-14','2020-08-15','2020-11-01','2020-11-11','2020-12-25','2020-12-26']

In [ ]:
public_holi21 = ["2021-01-01",'2021-04-02','2021-04-05','2021-05-01','2021-05-08','2021-05-13','2021-05-23','2021-05-24','2021-07-14','2021-08-15','2021-11-01','2021-11-11','2021-12-25','2021-12-26']

In [ ]:
dates_list = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in public_holi20]
df1["date"] = pd.to_datetime(df1["date"])

In [ ]:
df1['holiday'] = df1['date'].isin(dates_list)
df1.holiday = df1.holiday.replace([True, False], [1, 0])
df1.loc[:, "weekday"] = df1["date"].dt.weekday
df1['weekend'] = df1['weekday'].isin([5,6])
df1.weekend = df1.weekend.replace([True, False], [1, 0])
df1 = df1.drop('weekday', axis=1)
df1

In [ ]:
dates_list = [dt.datetime.strptime(date, '%Y-%m-%d').date() for date in public_holi21]
df2["date"] = pd.to_datetime(df2["date"])

In [ ]:
df2['holiday'] = df2['date'].isin(dates_list)
df2.holiday = df2.holiday.replace([True, False], [1, 0])
df2.loc[:, "weekday"] = df2["date"].dt.weekday
df2['weekend'] = df2['weekday'].isin([5,6])
df2.weekend = df2.weekend.replace([True, False], [1, 0])
df2 = df2.drop('weekday', axis=1)
df2

In [ ]:
df1.loc[:, "season"] = df1["date"].dt.month
df1.season = df1.season.replace([1,2,3,4,5,6,7,8,9,10,11,12], [3,3,3,0,0,1,1,1,1,2,2,3])
df1

In [ ]:
df2.loc[:, "season"] = df2["date"].dt.month
df2.season = df2.season.replace([1,2,3,4,5,6,7,8,9,10,11,12], [3,3,3,0,0,1,1,1,1,2,2,3])
df2

In [ ]:
df1['is_night'] = 0
df1.loc[(df1['hour'] < 8) | (df1['hour'] > 20), 'is_night'] = 1
df1

In [ ]:
df2['is_night'] = 0
df2.loc[(df2['hour'] < 8) | (df2['hour'] > 20), 'is_night'] = 1
df2

In [ ]:
weather = pd.concat([df1, df2], ignore_index=True)
weather = weather.drop(columns=['snow', 'wpgt','tsun','coco','Coco'])

In [ ]:
weather

In [ ]:
weather["hour"] = pd.to_timedelta(weather['hour'], unit='h')
weather['date'] = weather['date'] + weather['hour']
weather = weather.drop(columns=['hour'])
weather = weather.fillna(method='bfill')
weather

In [ ]:
lock1 = pd.date_range(start='10/30/2020', end='12/02/2020', freq='H').tolist()
lock2 = pd.date_range(start='04/03/2021', end='07/01/2021', freq='H').tolist()

In [ ]:
weather['lockdown1'] = weather['date'].isin(lock1)
weather.lockdown1 = weather.lockdown1.replace([True, False], [1, 0])
weather

In [ ]:
weather['lockdown2'] = weather['date'].isin(lock2)
weather.lockdown2 = weather.lockdown2.replace([True, False], [1, 0])
weather

In [1]:
#df.to_csv('external_data_covid.csv', date_format='%Y-%m-%d %H:%M:%S')

In [ ]:
train_merged = data.merge(weather, on='date')

In [ ]:
test_merged = data1.merge(weather, on='date')

In [ ]:
train_merged

In [ ]:
test_merged

In [ ]:
cols = ['bike_count','log_bike_count','latitude','longitude','temp','dwpt','rhum','prcp','wdir','wspd','pres','holiday','weekend','season','is_night']
def plot_distribution(london,columns,**kwargs ):
    """
    Function to plot a dataframe feature distribution.
    Input: 
    df - pandas DataFrame.
    columns - list of the df columns that should be plotted
    Output: multiple feature histograms
    
    Note: String columns should be dropped before passing to the function.
    """
    
    fig, axes = plt.subplots(ncols=len(columns),figsize=(30,5))
    for axs, col in zip(axes, columns):
        sns.distplot(london[col], ax=axs,**kwargs)
    fig.suptitle('Feature distribution')
    
plot_distribution(train_merged,cols, kde=False)

In [ ]:
train_merged_non_weekend = train_merged[train_merged['weekend'] == 0]
train_merged_non_weekend = train_merged_non_weekend.drop(['holiday','weekend'],axis=1)
train_merged_is_weekend = train_merged[train_merged['weekend'] == 1]
train_merged_is_weekend = train_merged_is_weekend.drop(['holiday','weekend'],axis=1)
train_merged_is_holiday = train_merged[train_merged['holiday'] == 1]
train_merged_is_holiday = train_merged_is_holiday.drop(['holiday','weekend'],axis=1)
train_merged_non_holiday = train_merged[train_merged['holiday'] == 0]
train_merged_non_holiday = train_merged_non_holiday.drop(['holiday','weekend'],axis=1)


In [ ]:
def hourly_plot(df,title):
    """
    Function for plotting bike shares by hour.
    input: 
    df - pandas dataframe 
    title - main title of the plot
    
    """
    df.groupby(by=df.date.dt.hour)['bike_count'].mean().plot()
    plt.title(title)
    plt.ylabel('count')
    plt.legend(['workdays','weekend','holidays'],loc=2, fontsize = 'medium')
    
hourly_plot(train_merged_non_weekend, 'Bike shares by hour')
hourly_plot(train_merged_is_weekend, 'Bike shares by hour')
hourly_plot(train_merged_is_holiday, 'Bike shares by hour')


In [3]:
corr = train_merged.corr()
corr_bike_count = corr['log_bike_count']
print(corr_bike_count)

NameError: name 'train_merged' is not defined

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(corr, linewidths=1)

In [ ]:
train_merged.hist(figsize=(20, 30))

In [ ]:
def add_hour(df):
    """
    The function is extracting the hour of a timestamp into a new column.
    Input: pandas DataFrame that contains 'timestamp' column
    """
    
    df['hour'] = df['date'].dt.hour

    return df

In [ ]:
def add_month(df):
    """
    The function is extracting the month of a timestamp into a new column.
    Input: pandas DataFrame that contains 'timestamp' column
    """
    
    df['month'] = df['date'].dt.month

    return df

In [ ]:
add_month(train_merged)
add_hour(train_merged)

In [ ]:
add_month(test_merged)
add_hour(test_merged)

In [ ]:
def add_encode(df, column, max_value):
    """
    The function is encoding time series cyclical features with sin and cos.
    Input: 
    ---------
    df - pandas DataFrame
    column - column name
    max_value - column max value
    Output: 
    -----------
    -same dataframe with _sin and _cos columns added
    """

    df[column + '_sin'] = np.sin(2 * np.pi * df[column] / max_value)
    df[column + '_cos'] = np.cos(2 * np.pi * df[column] / max_value)
    return df

In [ ]:
add_encode(train_merged,'hour',23)
add_encode(train_merged,'month',12)

In [ ]:
add_encode(test_merged,'hour',23)
add_encode(test_merged,'month',12)

In [ ]:
train_merged[train_merged['log_bike_count']>3].groupby(by='hour').count()

In [ ]:
def sns_hist(data,title):
    """"
    Function to plot seaborn histogram.
    """
    
    sns.distplot(data)#, fit='norm')
    plt.title(title)
    plt.ylabel('Density')

In [ ]:
sns_hist(train_merged[train_merged['is_night']==1]['log_bike_count'],"during the night.")

In [ ]:
sns_hist(train_merged[train_merged['is_night']==0]['log_bike_count'],"during the day.")

In [ ]:
train_merged['counter_installation_date'].unique()

In [ ]:
dm = dt.datetime.strptime('2020-11-29 00:00:00','%Y-%m-%d %H:%M:%S')
dm

In [ ]:
train_merged.loc[train_merged['counter_installation_date'] == dm]
#elm = train_merged.loc[train_merged['counter_installation_date'] < dm]
#elm

In [ ]:
elm.loc[elm['site_name'] == '254 rue de Vaugirard']

In [ ]:
elm['site_name'].unique().to_list()